# To Do
1. Account for: forwards vs reverse (maybe?) and library type 
2. Write to CSV? 
3. Graph data to quality check 
4. Download experiments (runs and metadata) using SRA toolkit and organize data folder structure using metadata 

We're using os and glob to do bash script stuff. 

In [49]:
import os
import glob

#ref folder name, ref genomes go here
ref_folder = 'references'
#os.system('mkdir '+ref_folder)

#data folder name, samples and runs go here
data = 'data'
#os.system('mkdir '+data)

# Organize Directorties Using Metadata
1. Match runs to samples using metadata
2. Create folder for each sample name
3. Move each run into corresponding sample folder
4. Quality to check to ensure that: reference genomes are in correct folder and data is in correct folder structure

data->samples->runs

Will do manually for now, but can think about automating this. 

# Building Index
Use salmon to build index. Download genomes from ensemble and store in refences folder. Will align to references in parallel. This is written to align to as many reference genomes as are put in the references folder. 

In [28]:
#grabs stuff and returns a list of them-used throughout
def grab_ref(ref_folder1):
    files=glob.glob(ref_folder1+'/*')
    #print(files)
    return(files)

#builds Salmon index from given file names, puts indexes into folder, INDEX_references 
def salmon_index(genome):
    os.system('salmon index -t '+str(genome)+' -i INDEX_'+str(genome))
    #print('salmon '+ genome)
    return 

#call grab_ref() and salmon_index() to build index for each reference genome
for i in grab_ref(ref_folder):
    salmon_index(i)

# Get Quants for Each Reference Genome on a Per Sample Basis
We want all of the runs from each sample to be fed into salmon at one time. This is currently robust to number of runs, but is not robust to library type or paired vs unpaired. 

In [48]:
#need to check salmon output path and need to see about paired reads
for y in glob.glob('INDEX_'+ref_folder+'/*'):
    print(y)
    for j in grab_ref(data):
        #print(j)
        runs = grab_ref(j)
        #print(runs)
        if len(runs)==1:
            print(runs)
            #os.system('salmon quant -t '+y+' -l -a '+runs[0]+salmon_quant')
        else:
            string=''
            for z in runs:
                string+= ' '+ z
            #print(string)
            #os.system('salmon quant -t '+y+' -l -a '+string+' o salmon_quant')

INDEX_references/Pseudomonas_aeruginosa_UCBPP-PA14_109.ffn.gz
INDEX_references/Pseudomonas_aeruginosa_PAO1_107.ffn.gz


# Storing Output
Not sure yet what the tail end of this will be like.

# Graphing Data to Quality Check
Also not sure what this should look like yet. 